# Lab04 — Bayes's theorem example with a Python class

## A simple class:  Production line failure problem

Here we do a simple discrete-hypotheses, discrete-data problem much like the coin flipping problem done in Lec04 (with three hypotheses about $\alpha$, the probability for heads or success).

We computed probabilities numerically for that problem in a Jupyter notebook, using (global) variables to store problem info (priors, data), and a function to compute likelihoods. To treat different cases, we modified global variables, and then re-ran the whole notebook, comparing new results to previous ones by memory.

Here we'll solve a related problem, but implement the calculations using a Python class. Class instances will serve as isolated "containers" that let us handle multiple cases all together in a notebook.

This exercise is meant to illustrate **using Python classes**. It also demonstrates a good programming practice: using **unit tests** (see [Test-driven development - Wikipedia](https://en.wikipedia.org/wiki/Test-driven_development) to learn about a popular software development practice using unit tests).

### Problem setup

Consider a production line producing batches of [widgets](https://en.wikipedia.org/wiki/Widget_(economics)) (given recent past events, I'm tempted to use fidget spinners instead of widgets, but I'll resist the temptation!).

When the production line works properly (the *nominal* state), it produces widgets with a small defect rate, $\alpha$ (known from past measurements).

Sometimes the production line suffers an internal partial failure on startup, and produces widgets with a larger defect rate, $\beta$ (also known from past measurements).

Historical data indicates that the probability of failure on startup is $f$ (also known).

The production line is started and produces a batch of $N$ widgets, $n_d$ of them defective.

**_What is the posterior probability that the production line operated in the failure mode?_**

### Likelihood function

You may have noticed that the problem isn't fully specified. In particular, it's not obvious what rules governed the run producing the data. Here are two possibilities:

* Run the line to produce some target number of widgets, $N$. Count the number of defects among those $N$ widgets.
* Run the line to meet an order for $m$ good widgets, testing widgets as the line runs.

These two production strategies correspond to different likelihood functions:

* For the first, $N$ is fixed, and $n_f$ is a random number. You can probably see that the sampling distribution for $n_f$ is a multinomial distribution.
* For the second, $m$ is fixed, and $N$ is a random number; the number of failures is $n_f = N-m$.

What's the sampling distribution in the second case? For the purpose of determining whether the production line failed, does it matter which strategy was used?

### Posterior probabilities

Compute the posterior probabilities for $H_0 = $ nominal state and $H_1 =$ failure mode by filling out a probability table like this:

| Hypothesis | Prior | Likelihood | Prior $\times$ Likelihood | Posterior |
|------------|-------|------------|---------------------------|-----------|
| H_0        |       |            |                           |           |
| H_1        |       |            |                           |           |
| Sum        | 1     | NA         |                           |     1     |

(You'll probably need more room than that—do it on paper or a whiteboard.)

The formulas aren't very illuminating, so let's compute the probabilities numerically for various cases.

The accompanying Python source file, `machine_failure.py`, can be used as either a script (run at the command line) or a module (imported into other Python code, or a notebook). It defines a Python class doing the necessary calculations in two ways:

* Computing probabilities directly;
* Computing logarithms of probabilities for intermediate steps.

Ideally, these approaches should always give the same results.  But for large samples Bayesian calculations can involve summing and computing ratios of very small probabilities, smaller than floating point precision can represent. So most Bayesian codes work with logarithms of probabilities as much as possible. This won't be important for the cases we'll treat here, but it's good to get used to thinking about log-probabilities for statistical computation.

Have a look at the file.  It defines a Python class, `MachineFailure`, and some test functions.

At the end, it uses the class to run some example cases. These are in a special `if` block indicating these are meant to be computed when the file is run as a script at the command line (with a `python` or `ipython` command).  You can also run scripts in a Jupyter notebook: 

In [ ]:
%run machine_failure.py

The file also includes some functions with names starting with `test_`. These are **unit tests**, simple cases meant to test small pieces of functionality. Several Python tools exist that handles such tests.  Two popular ones that are largely interoperable are `nose` and `pytest` (`pytest` is newer and has pretty much become the popular standard). Both tools provide terminal commands that will gather and run tests in script (note: in the current Anaconda Py-3.7 distribution, `pytest` gives a warning from a bug in a SciPy test utility module):

In [ ]:
!nosetests machine_failure.py

In [ ]:
!pytest machine_failure.py

We can also import `machine_failure` as a module. Here we'll import it and add the `MachineFailure` class to the notebook's namespace (actually, having run the script, it's already here, but this is what you'd do if you didn't want to run the script first).

In [ ]:
from machine_failure import MachineFailure

Use introspection for a reminder of how to use the class:

In [ ]:
MachineFailure?

Prepare for cases where the production line fails 10% of the time, with nominal and failure mode defect rates of 1% and 40% (these values are from an example in a popular textbook; the script uses them for a test case, since we know the answer from the book):

In [ ]:
mf = MachineFailure(.01, .4, .1)

The `update_data` method lets us (cumulatively) introduce $(N,n_f)$ data values, and the `post_failed` method computes the probability the production line is in failure mode:

In [ ]:
mf.update_data(6, 2)  # N, n_f
mf.post_failed()

So we're pretty sure the production line failed, even with that small sample and even though failure is fairly unlikely, mainly because the failure mode defect rate is so high that it gets obvious quickly when the production line has failed.

Lets look at how the probability changes as the sample size grows, when the fraction of failures is fixed near its expected value.

In [ ]:
mf = MachineFailure(.01, .4, .1)

def const_freq_update(mf, f, N):
    """
    Using a MachineFailure instance, add some data with a specified fraction
    of failures, and return the total sample size and the failure mode probability.
    """
    mf.update_data(N, int(f*N))
    return mf.N, mf.post_failed()

# Gather results adding batches of different sizes using a list comprehension:
N_p = [const_freq_update(mf, .4, N) for N in [5, 5, 10, 20, 50, 100]]

# That's a list; make it an array, and show it in the notebook:
N_p = array(N_p)
N_p

In [ ]:
# Plot those data.

from matplotlib.pyplot import *
%matplotlib inline

plot(N_p[:,0], N_p[:,1], 'bo-')
xlabel('N')
ylabel(r'$p_{\rm failed}$, const freq.')

Now try a case where the nominal defect rate is already pretty high, so it gets harder to distinguish the nominal and failure modes.

In [ ]:
mf = MachineFailure(.3, .4, .1)
N_p = array([const_freq_update(mf, .4, N) for N in [5, 5, 10, 20, 50, 100]])
N_p

In [ ]:
plot(N_p[:,0], N_p[:,1], 'bo-')
xlabel('N')
ylabel(r'$p_{\rm failed}$, const freq.')